### Interacción entre pandas y SQL

In [26]:
import pandas as pd
import sqlite3

from funciones_sql import esquema_tablas, vista_rapida_query

In [11]:
# Conectar a la base de datos
conn = sqlite3.connect('../yeast.sqlite')

In [12]:
cursor = conn.cursor()

In [13]:
esquema_tablas(cursor)


Tabla: sqlite_sequence
  - name () 
  - seq () 

Tabla: Genes
  - id (INTEGER) PRIMARY KEY
  - name (TEXT) 

Tabla: locations
  - id (INTEGER) PRIMARY KEY
  - location (TEXT) 

Tabla: Molecular
  - id (INTEGER) PRIMARY KEY
  - function (TEXT) 

Tabla: Bioprocess
  - id (INTEGER) PRIMARY KEY
  - process (TEXT) 

Tabla: Member
  - gene_id (INTEGER) PRIMARY KEY
  - location_id (INTEGER) PRIMARY KEY
  - function_id (INTEGER) PRIMARY KEY
  - bioprocess_id (INTEGER) PRIMARY KEY

Tabla: Treatment
  - id (INTEGER) PRIMARY KEY
  - condition (TEXT) 

Tabla: Description
  - id (INTEGER) PRIMARY KEY
  - meaning (TEXT) 

Tabla: Relation
  - condition_id (INTEGER) PRIMARY KEY
  - meaning_id (INTEGER) PRIMARY KEY

Tabla: Expression
  - gene_id (INTEGER) 
  - transcripts (INTEGER) 
  - treatment_id (INTEGER) 
  - location_id (INTEGER) 
  - meaning_id (INTEGER) 
  - function_id (INTEGER) 
  - bioprocess_id (INTEGER) 


In [21]:
# Leer la tabla 'empleados' como un DataFrame
df = pd.read_sql_query("SELECT * FROM Description LIMIT 10;", conn)
df

,id,meaning
0,1,wildtype wildtype 1
1,2,wildtype wildtype 2
2,3,itc1 itc1-1_dUTP
3,5,swr1 swr1 mutant
4,7,tet-STH1 <not provided>
5,9,tet-INO80 tet-ino80
6,11,tet-control strain tet-control
7,13,Strain5 phenol lysis
8,15,Strain6 phenol lysis
9,19,Strain7 phenol lysis


In [10]:
conn.commit()
conn.close()

¿Cómo puedo convertir un archivo csv a sqlite?

In [23]:
df = pd.read_csv('../diabetes_dataset.csv')

In [24]:
# Conectar a SQLite
conn = sqlite3.connect('diabetes.sqlite')

# Escribir el DataFrame a una nueva tabla (o reemplazarla si existe)
df.to_sql('diabetes', conn, if_exists='replace', index=False)

conn.close()

Chequemos si la tabla fue transformada:

In [31]:
conn = sqlite3.connect('diabetes.sqlite')
cursor = conn.cursor()

In [35]:
vista_rapida_query("SELECT * FROM diabetes", conn)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [41]:
conn.commit()
conn.close()

### Unas cuestiones más de SQL...

In [42]:
conn = sqlite3.connect('../covid_database.sqlite')
cursor = conn.cursor()

In [43]:
esquema_tablas(cursor)


Tabla: country_wise_latest
  - Country/Region (TEXT) 
  - Confirmed (INTEGER) 
  - Deaths (INTEGER) 
  - Recovered (INTEGER) 
  - Active (INTEGER) 
  - New cases (INTEGER) 
  - New deaths (INTEGER) 
  - New recovered (INTEGER) 
  - Deaths / 100 Cases (REAL) 
  - Recovered / 100 Cases (REAL) 
  - Deaths / 100 Recovered (REAL) 
  - Confirmed last week (INTEGER) 
  - 1 week change (INTEGER) 
  - 1 week % increase (REAL) 
  - WHO Region (TEXT) 

Tabla: covid_19_clean_complete
  - Province/State (TEXT) 
  - Country/Region (TEXT) 
  - Lat (REAL) 
  - Long (REAL) 
  - Date (TEXT) 
  - Confirmed (INTEGER) 
  - Deaths (INTEGER) 
  - Recovered (INTEGER) 
  - Active (INTEGER) 
  - WHO Region (TEXT) 

Tabla: full_grouped
  - Date (TEXT) 
  - Country/Region (TEXT) 
  - Confirmed (INTEGER) 
  - Deaths (INTEGER) 
  - Recovered (INTEGER) 
  - Active (INTEGER) 
  - New cases (INTEGER) 
  - New deaths (INTEGER) 
  - New recovered (INTEGER) 
  - WHO Region (TEXT) 

Tabla: worldometer_data
  - Country/Re

In [49]:
vista_rapida_query("""SELECT * FROM covid_19_clean_complete;""", conn)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,None,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,None,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,None,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,None,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,None,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [60]:
query = """ 
SELECT *
FROM covid_19_clean_complete
FULL JOIN country_wise_latest ON covid_19_clean_complete."Country/Region" = country_wise_latest."Country/Region";
"""

In [61]:
vista_rapida_query(query, conn)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region,...,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,None,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean,...,106,10,18,3.50,69.49,5.04,35526,737,2.07,Eastern Mediterranean
1,None,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe,...,117,6,63,2.95,56.25,5.25,4171,709,17.00,Europe
2,None,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa,...,616,8,749,4.16,67.34,6.17,23691,4282,18.07,Africa
3,None,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe,...,10,0,0,5.73,88.53,6.48,884,23,2.60,Europe
4,None,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa,...,18,1,0,4.32,25.47,16.94,749,201,26.84,Africa


Podemos elegir columnas individuales de tablas distintas? La respuesta es si.

Problema: Obtener el número de muertes promedio por país.

In [93]:
query = """ 
SELECT covid_complete."Country/Region", covid_complete."WHO Region", AVG(country."Deaths / 100 Cases")
FROM covid_19_clean_complete AS covid_complete
RIGHT JOIN country_wise_latest AS country ON covid_complete."Country/Region" = country."Country/Region"
GROUP BY covid_complete."Country/Region"
ORDER BY country."Deaths / 100 Cases" DESC;
"""

In [94]:
vista_rapida_query(query, conn, limite = 1000)

,Country/Region,WHO Region,"AVG(country.""Deaths / 100 Cases"")"
0,Yemen,Eastern Mediterranean,28.56
1,United Kingdom,Europe,15.19
2,Belgium,Europe,14.79
3,Italy,Europe,14.26
4,France,Europe,13.71
...,...,...,...
182,Fiji,Western Pacific,0.00
183,Eritrea,Africa,0.00
184,Dominica,Americas,0.00
185,Cambodia,Western Pacific,0.00


Supongamos ahora que quiero más estadísticos.

In [117]:
query = """ 
SELECT covid_complete."Country/Region", covid_complete."WHO Region", AVG(country."Deaths / 100 Cases") AS avg_death_cases, AVG(country."Deaths / 100 Recovered") AS avg_death_recovered, AVG(country."Recovered / 100 Cases") AS avg_recovered_cases
FROM covid_19_clean_complete AS covid_complete
RIGHT JOIN country_wise_latest AS country ON covid_complete."Country/Region" = country."Country/Region"
GROUP BY covid_complete."Country/Region"
ORDER BY country."Deaths / 100 Cases" DESC;
"""

In [118]:
vista_rapida_query(query, conn, limite = 1000)

,Country/Region,WHO Region,avg_death_cases,avg_death_recovered,avg_recovered_cases
0,Yemen,Eastern Mediterranean,28.56,57.98,49.26
1,United Kingdom,Europe,15.19,3190.26,0.48
2,Belgium,Europe,14.79,56.28,26.27
3,Italy,Europe,14.26,17.68,80.64
4,France,Europe,13.71,37.20,36.86
...,...,...,...,...,...
182,Fiji,Western Pacific,0.00,0.00,66.67
183,Eritrea,Africa,0.00,0.00,72.08
184,Dominica,Americas,0.00,0.00,100.00
185,Cambodia,Western Pacific,0.00,0.00,65.04


Esta tabla sola ya es bastante grande... Podemos utilizar para hacer queries? Por supuesto que si. Estas son llamadas subqueries.

In [135]:
query_with_subquery = """ 
SELECT subquery_table."WHO Region", AVG(subquery_table.avg_death_cases)
FROM (SELECT covid_complete."Country/Region", covid_complete."WHO Region", AVG(country."Deaths / 100 Cases") AS avg_death_cases, AVG(country."Deaths / 100 Recovered") AS avg_death_recovered, AVG(country."Recovered / 100 Cases") AS avg_recovered_cases
    FROM covid_19_clean_complete AS covid_complete
    RIGHT JOIN country_wise_latest AS country ON covid_complete."Country/Region" = country."Country/Region"
    GROUP BY covid_complete."Country/Region"
    ORDER BY country."Deaths / 100 Cases" DESC) AS subquery_table
GROUP BY subquery_table."WHO Region";
"""

In [136]:
vista_rapida_query(query_with_subquery, conn)

,WHO Region,AVG(subquery_table.avg_death_cases)
0,Africa,2.306458
1,Americas,3.052571
2,Eastern Mediterranean,3.563182
3,Europe,4.198393
4,South-East Asia,1.296000
